<a href="https://github.com/zia207/python-colab/blob/main/NoteBook/Python_for_Beginners/001-03-02-data-wrangling-pyjanitor-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://drive.google.com/uc?export=view&id=1IFEWet-Aw4DhkkVe1xv_2YYqlvRe9m5_)

# 3.2 Data Wrangling with pyjanitor

`pyjanitor` is a Python library that extends pandas, providing intuitive functions for data cleaning and preprocessing. Inspired by R's janitor package, it simplifies common data manipulation tasks with clear, chainable methods. Key features include cleaning column names, handling missing values, filtering, and grouping data, making it ideal for streamlining data preparation workflows. It integrates seamlessly with pandas DataFrames, offering a user-friendly API for both beginners and advanced users.

## Prerequisites

Install the required packages:

In [12]:
import importlib.util
import sys

# List of required packages
packages = ['pandas', 'janitor']

# Check and install missing packages
for package in packages:
    if not importlib.util.find_spec(package):
        try:
            import pip
            pip.main(['install', package])
        except ImportError:
            print(f"Failed to install {package}. Pip is not available.")


# Import packages
import pandas as pd
import janitor

In [9]:
# Verify package availability
for package in packages:
    print(f"{package} installed: {bool(importlib.util.find_spec(package))}")


pandas installed: True
janitor installed: True


## Data Wrangling

We will create some "messy" data and clean it using pyjanitor functions, including `clean_names()`, `remove_empty()`, `find_duplicates()`, and `drop_constant_columns()`. Later, we will clean a real dataset to demonstrate a complete workflow.

### clean_names()

The `clean_names()` function standardizes column names by converting them to snake_case and removing special characters.

In [10]:
# Create a DataFrame with messy column names
df = pd.DataFrame({
    "Column One": [1, 2, 3, 4, 5],
    "Column Two!!": [6, 7, 8, 9, 10],
    "Column Three $": [11, 12, 13, 14, 15],
    "%Column four": [11, 12, 13, 14, 15]
})

print("Original DataFrame:")
print(df.head())

# Clean column names
df_cleaned = df.clean_names()
print("\nDataFrame with cleaned names:")
print(df_cleaned.head())

Original DataFrame:
   Column One  Column Two!!  Column Three $  %Column four
0           1             6              11            11
1           2             7              12            12
2           3             8              13            13
3           4             9              14            14
4           5            10              15            15

DataFrame with cleaned names:
   column_one  column_two!!  column_three_$  %column_four
0           1             6              11            11
1           2             7              12            12
2           3             8              13            13
3           4             9              14            14
4           5            10              15            15


### get_dupes()

The `get_dupes()` function identifies duplicate rows in a DataFrame based on specified columns.

In [11]:
# Create a DataFrame with duplicate rows
df = pd.DataFrame({
    "Column One": [1, 2, 3, 1],
    "Column Two": ["A", "B", "C", "A"]
})

print("Original DataFrame:")
print(df)

# Find duplicates using pyjanitor's get_dupes function
duplicates = df.get_dupes(columns=["Column One", "Column Two"])

print("\nDuplicate rows:")
print(duplicates)

Original DataFrame:
   Column One Column Two
0           1          A
1           2          B
2           3          C
3           1          A

Duplicate rows:
   Column One Column Two
0           1          A
3           1          A


/home/zia207/.local/lib/python3.10/site-packages/janitor/utils.py:367: DeprecationWarning: columns is deprecated; use column_names
  warn(


### remove_empty()

The  `remove_empty()` function removes rows or columns that are entirely empty or contain only missing values.

In [27]:
# Create a DataFrame with empty rows and columns
df = pd.DataFrame({
    "x": [1, None, 4],
    "y": [None, None, 3],
    "z": [None, None, None]
})

print("Original DataFrame:")
print(df)

# Remove empty rows and columns
df_cleaned = df.remove_empty()
print("\nDataFrame after removing empty rows/columns:")
print(df_cleaned)

Original DataFrame:
     x    y     z
0  1.0  NaN  None
1  NaN  NaN  None
2  4.0  3.0  None

DataFrame after removing empty rows/columns:
     x    y
0  1.0  NaN
1  4.0  3.0


##  Cleaning Messy Data

Now we will clean on very messy data using some functions of janitor packages. We will use [Lung Cancer Mortality data](https://www.dropbox.com/s/ovm7dc2szax6kcz/USA_LBC_Data.csv?dl=0).


### Data

The raw file has **one column** containing all fields separated by `|`.

In [28]:
# Load the dataset
url = "https://github.com/zia207/r-colab/raw/main/Data/R_Beginners/USA_LBC_Data_raw.csv"
df = pd.read_csv(url)

print("Original DataFrame:")
print(df.head())

Original DataFrame:
   Lung Cancer Moratlity Rates and Risk in USA, Data Provider: Zia Ahmed Unnamed: 1       Unnamed: 2      Unnamed: 3   Unnamed: 4   Unnamed: 5 Unnamed: 6      Unnamed: 7          Unnamed: 8 Unnamed: 9 Unnamed: 10 Unnamed: 11 Unnamed: 12 Unnamed: 13 Unnamed: 14 Unnamed: 15  Unnamed: 16 Unnamed: 17 Unnamed: 18 Unnamed: 19      Unnamed: 20 Unnamed: 21 Unnamed: 22       Unnamed: 23         Unnamed: 24      Unnamed: 25
0                                          REGION_ID                          STATE           County  Empty Column 1            X            Y       Fips  Empty_Column 2  LCB Mortality Rate    Smoking      PM  25         NO2         SO2       Ozone      Pop 65   Pop Black  Pop Hipanic   Pop White   Education   Poverty %  Income Equality   Uninsured         DEM  Radon Zone Class         Urban Rural  Coal Production
1                                                  3                        Alabama   Baldwin County             NaN  789777.5039  884557.0795  

### Promote Row to Column Names

The dataset may have descriptive text in the first row instead of proper column headers. We can use row_to_names() to set the first row as column names and remove it from the data.

In [29]:
# Promote first row to column names
df_01 = df.row_to_names(row_number=0, remove_row=True, remove_rows_above=True)
print("\nDataFrame after row_to_names:")
print(df_01.head())


DataFrame after row_to_names:
  REGION_ID    STATE           County Empty Column 1            X            Y  Fips Empty_Column 2 LCB Mortality Rate Smoking PM  25     NO2       SO2  Ozone Pop 65 Pop Black Pop Hipanic Pop White Education Poverty % Income Equality Uninsured     DEM Radon Zone Class         Urban Rural Coal Production
1         3  Alabama   Baldwin County            NaN  789777.5039  884557.0795  1003            NaN               48.1    20.8   7.89  0.7939  0.035343  39.79   19.5      9.24        4.54     83.06        66     13.14             4.5     13.34   36.78           Zone-3  Medium/small metro              No
2         3  Alabama    Butler County            NaN  877731.5725   1007285.71  1013            NaN               38.3      26   8.46  0.6344    0.0135  38.31     19     43.94        1.26     52.64        38     26.14             5.1     12.74  111.70           Zone-3            Nonmetro              No
3         3  Alabama    Butler County            NaN  

/home/zia207/.local/lib/python3.10/site-packages/janitor/utils.py:367: DeprecationWarning: row_number is deprecated; use row_numbers
  warn(
/home/zia207/.local/lib/python3.10/site-packages/janitor/utils.py:367: DeprecationWarning: remove_row is deprecated; use remove_rows
  warn(



### Remove Empty Rows and Columns

Next, we remove any empty rows or columns using `remove_empty()`.

In [30]:
# Remove empty rows and columns
df_02 = df_01.remove_empty()
print("\nDataFrame after removing empty rows/columns:")
print(df_02.head())


DataFrame after removing empty rows/columns:
  REGION_ID    STATE           County            X            Y  Fips LCB Mortality Rate Smoking PM  25     NO2       SO2  Ozone Pop 65 Pop Black Pop Hipanic Pop White Education Poverty % Income Equality Uninsured     DEM Radon Zone Class         Urban Rural Coal Production
0         3  Alabama   Baldwin County  789777.5039  884557.0795  1003               48.1    20.8   7.89  0.7939  0.035343  39.79   19.5      9.24        4.54     83.06        66     13.14             4.5     13.34   36.78           Zone-3  Medium/small metro              No
1         3  Alabama    Butler County  877731.5725   1007285.71  1013               38.3      26   8.46  0.6344    0.0135  38.31     19     43.94        1.26     52.64        38     26.14             5.1     12.74  111.70           Zone-3            Nonmetro              No
2         3  Alabama    Butler County  877731.5725   1007285.71  1013               38.3      26   8.46  0.6344    0.0135  38.31 

### Clean Column Names

We standardize column names using clean_names().

In [31]:
# Clean column names
df_03 = df_02.clean_names()
print("\nDataFrame with cleaned column names:")
print(df_03.head())


DataFrame with cleaned column names:
  region_id    state           county            x            y  fips lcb_mortality_rate smoking pm_25     no2       so2  ozone pop_65 pop_black pop_hipanic pop_white education poverty_% income_equality uninsured     dem radon_zone_class         urban_rural coal_production
0         3  Alabama   Baldwin County  789777.5039  884557.0795  1003               48.1    20.8  7.89  0.7939  0.035343  39.79   19.5      9.24        4.54     83.06        66     13.14             4.5     13.34   36.78           Zone-3  Medium/small metro              No
1         3  Alabama    Butler County  877731.5725   1007285.71  1013               38.3      26  8.46  0.6344    0.0135  38.31     19     43.94        1.26     52.64        38     26.14             5.1     12.74  111.70           Zone-3            Nonmetro              No
2         3  Alabama    Butler County  877731.5725   1007285.71  1013               38.3      26  8.46  0.6344    0.0135  38.31     19     4

### Convert Data Types

The dataset may have incorrect data types (e.g., numeric columns stored as strings). We use `convert_to_numeric()` and pandas' `astype()` to fix this. Assuming columns 4 to 21 are numeric and 22 to 24 are categorical 

In [ ]:
# Assuming df_03 is the DataFrame after cleaning names
# Convert columns 4 to 21 to numeric using pandas' pd.to_numeric
# Convert columns 22 to 24 to categorical
df_04 = df_03.copy()  # Create a copy to avoid modifying the original
for col in df_03.columns[3:21]:
    df_04[col] = pd.to_numeric(df_03[col], errors="coerce")
df_04 = df_04.astype({col: 'category' for col in df_03.columns[21:24]})

print("\nDataFrame with corrected data types:")
print(df_04.dtypes)


DataFrame with corrected data types:
region_id               object
state                   object
county                  object
x                      float64
y                      float64
fips                     int64
lcb_mortality_rate     float64
smoking                float64
pm_25                  float64
no2                    float64
so2                    float64
ozone                  float64
pop_65                 float64
pop_black              float64
pop_hipanic            float64
pop_white              float64
education                int64
poverty_%              float64
income_equality        float64
uninsured              float64
dem                    float64
radon_zone_class      category
urban_rural           category
coal_production       category
dtype: object


### Remove Duplicates

We check for and remove duplicate rows based on a key column (e.g., fips).

In [36]:

# Check for duplicates using pyjanitor's get_dupes
duplicates = df_04.get_dupes(columns="fips")
print("\nDuplicate rows based on 'fips':")
print(duplicates)

# Remove duplicates using pandas' drop_duplicates
df_05 = df_04.drop_duplicates(subset="fips", keep="first")
print("\nDataFrame after removing duplicates:")
print(df_05.head())


Duplicate rows based on 'fips':
   region_id     state           county            x             y  fips  lcb_mortality_rate  smoking  pm_25     no2       so2  ozone  pop_65  pop_black  pop_hipanic  pop_white  education  poverty_%  income_equality  uninsured     dem radon_zone_class urban_rural coal_production
1          3   Alabama    Butler County  877731.5725  1.007286e+06  1013                38.3     26.0   8.46  0.6344  0.013500  38.31    19.0      43.94         1.26      52.64         38      26.14              5.1      12.74  111.70           Zone-3    Nonmetro              No
2          3   Alabama    Butler County  877731.5725  1.007286e+06  1013                38.3     26.0   8.46  0.6344  0.013500  38.31    19.0      43.94         1.26      52.64         38      26.14              5.1      12.74  111.70           Zone-3    Nonmetro              No
11         3   Alabama  Escambia County  839009.4497  9.335802e+05  1053                58.3     25.3   8.08  0.5742  0.025900 

/home/zia207/.local/lib/python3.10/site-packages/janitor/utils.py:367: DeprecationWarning: columns is deprecated; use column_names
  warn(


###  Combine All Steps

We can chain all operations using pandas' method chaining for a clean workflow.

In [38]:
# Cleaning pipeline
df_clean = df.copy()
df_clean = (df_clean
            .row_to_names(row_number=0, remove_row=True, remove_rows_above=True)
            .remove_empty()
            .clean_names()
)

# Convert columns 4 to 21 to numeric
numeric_cols = df_clean.columns[3:21]
for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

# Convert columns 22 to 24 to categorical and remove duplicates
df_clean = (df_clean
            .astype({col: 'category' for col in df_clean.columns[21:24]})
            .drop_duplicates(subset="fips", keep="first")
)

print("\nFully cleaned DataFrame:")
print(df_clean.head())
print("\nData types:")
print(df_clean.dtypes)

# Check for duplicates
duplicates = df_clean.get_dupes(columns="fips")
print("\nDuplicate rows based on 'fips':")
print(duplicates)


Fully cleaned DataFrame:
  region_id    state           county            x             y  fips  lcb_mortality_rate  smoking  pm_25     no2       so2  ozone  pop_65  pop_black  pop_hipanic  pop_white  education  poverty_%  income_equality  uninsured     dem radon_zone_class         urban_rural coal_production
0         3  Alabama   Baldwin County  789777.5039  8.845571e+05  1003                48.1     20.8   7.89  0.7939  0.035343  39.79    19.5       9.24         4.54      83.06         66      13.14              4.5      13.34   36.78           Zone-3  Medium/small metro              No
1         3  Alabama    Butler County  877731.5725  1.007286e+06  1013                38.3     26.0   8.46  0.6344  0.013500  38.31    19.0      43.94         1.26      52.64         38      26.14              5.1      12.74  111.70           Zone-3            Nonmetro              No
3         3  Alabama  Chambers County  984214.6861  1.148649e+06  1017                49.6     25.1   8.87  0.8442  

/home/zia207/.local/lib/python3.10/site-packages/janitor/utils.py:367: DeprecationWarning: row_number is deprecated; use row_numbers
  warn(
/home/zia207/.local/lib/python3.10/site-packages/janitor/utils.py:367: DeprecationWarning: remove_row is deprecated; use remove_rows
  warn(
/home/zia207/.local/lib/python3.10/site-packages/janitor/utils.py:367: DeprecationWarning: columns is deprecated; use column_names
  warn(


## Some Important Functions

### **`conditional_join`**

Unlike `pd.merge`, which only handles equi-joins (equality conditions), `conditional_join` supports joins based on inequality operators (`<`, `<=`, `>`, `>=`, `==`, `!=`) or combinations of both.

>df.conditional_join(right_df, (left_column, right_column, operator), ...)

Where:

-   `df`: The left DataFrame

-   `right_df`: The right DataFrame to join with

-   `(left_column, right_column, operator)`: A tuple defining the join condition

-   `operator`: One of `==`, `!=`, `<=`, `<`, `>=`, `>`

### **Range Join**

Let's say we have two datasets:

-   `employees`: Contains employee IDs and their salary ranges

-   `jobs`: Contains job positions with required salary ranges

We want to find which jobs each employee qualifies for based on their salary falling within the job's range.

In [5]:
# Create sample data
employees = pd.DataFrame({
    'employee_id': [1, 2, 3, 4],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana'],
    'salary': [50000, 75000, 90000, 60000]
})

jobs = pd.DataFrame({
    'job_title': ['Junior Developer', 'Senior Developer', 'Manager', 'Director'],
    'min_salary': [40000, 60000, 80000, 100000],
    'max_salary': [60000, 85000, 120000, 150000]
})

print("Employees:")
print(employees)
print("\nJobs:")
print(jobs)

Employees:
   employee_id     name  salary
0            1    Alice   50000
1            2      Bob   75000
2            3  Charlie   90000
3            4    Diana   60000

Jobs:
          job_title  min_salary  max_salary
0  Junior Developer       40000       60000
1  Senior Developer       60000       85000
2           Manager       80000      120000
3          Director      100000      150000


In [6]:
# Join employees with jobs where employee salary is between min and max salary
result = employees.conditional_join(
    jobs,
    ('salary', 'min_salary', '>='),  # employee.salary >= job.min_salary
    ('salary', 'max_salary', '<=')   # employee.salary <= job.max_salary
)

print("\nEmployees qualified for jobs:")
print(result)


Employees qualified for jobs:
   employee_id     name  salary         job_title  min_salary  max_salary
0            1    Alice   50000  Junior Developer       40000       60000
1            2      Bob   75000  Senior Developer       60000       85000
2            3  Charlie   90000           Manager       80000      120000
3            4    Diana   60000  Junior Developer       40000       60000
4            4    Diana   60000  Senior Developer       60000       85000


### **Controlling Output Columns**

Often, you don't need all columns from both DataFrames. Use `df_columns` and `right_columns` to select specific ones:

In [7]:
# Only keep employee info and job title
result_slim = employees.conditional_join(
    jobs,
    ('salary', 'min_salary', '>='), 
    ('salary', 'max_salary', '<='),
    df_columns=['employee_id', 'name', 'salary'],  # Columns from employees
    right_columns='job_title'  # Only job_title from jobs
)

print("\nSimplified result - only employee info and job title:")
print(result_slim)


Simplified result - only employee info and job title:
   employee_id     name  salary         job_title
0            1    Alice   50000  Junior Developer
1            2      Bob   75000  Senior Developer
2            3  Charlie   90000           Manager
3            4    Diana   60000  Junior Developer
4            4    Diana   60000  Senior Developer


### **Using Different Join Types**

You can specify different types of joins similar to `pd.merge`:

In [8]:
# Left join: Include all employees, even if they don't qualify for any job
result_left = employees.conditional_join(
    jobs,
    ('salary', 'min_salary', '>='), 
    ('salary', 'max_salary', '<='),
    how='left'
)

print("\nLeft join - all employees included:")
print(result_left)


Left join - all employees included:
   employee_id     name  salary         job_title  min_salary  max_salary
0            1    Alice   50000  Junior Developer       40000       60000
1            2      Bob   75000  Senior Developer       60000       85000
2            3  Charlie   90000           Manager       80000      120000
3            4    Diana   60000  Junior Developer       40000       60000
4            4    Diana   60000  Senior Developer       60000       85000


### Getting First or Last Match Only
When multiple matches exist (like Charlie qualifying for two jobs), you might only want one:

In [9]:
# Get only the first matching job for each employee
result_first = employees.conditional_join(
    jobs,
    ('salary', 'min_salary', '>='), 
    ('salary', 'max_salary', '<='),
    keep='first'
)

print("\nFirst match only:")
print(result_first)

# Get only the last matching job for each employee
result_last = employees.conditional_join(
    jobs,
    ('salary', 'min_salary', '>='), 
    ('salary', 'max_salary', '<='),
    keep='last'
)

print("\nLast match only:")
print(result_last)


First match only:
   employee_id     name  salary         job_title  min_salary  max_salary
0            1    Alice   50000  Junior Developer       40000       60000
1            2      Bob   75000  Senior Developer       60000       85000
2            3  Charlie   90000           Manager       80000      120000
3            4    Diana   60000  Junior Developer       40000       60000

Last match only:
   employee_id     name  salary         job_title  min_salary  max_salary
0            1    Alice   50000  Junior Developer       40000       60000
1            2      Bob   75000  Senior Developer       60000       85000
2            3  Charlie   90000           Manager       80000      120000
3            4    Diana   60000  Senior Developer       60000       85000


### Adding an Indicator Column
To understand the source of each row (especially useful in outer joins):

In [10]:
# Outer join with indicator
result_outer = employees.conditional_join(
    jobs,
    ('salary', 'min_salary', '>='), 
    ('salary', 'max_salary', '<='),
    how='outer',
    indicator=True
)

print("\nOuter join with indicator column:")
print(result_outer)


Outer join with indicator column:
   employee_id     name   salary         job_title  min_salary  max_salary  \
0          1.0    Alice  50000.0  Junior Developer       40000       60000   
1          2.0      Bob  75000.0  Senior Developer       60000       85000   
2          3.0  Charlie  90000.0           Manager       80000      120000   
3          4.0    Diana  60000.0  Junior Developer       40000       60000   
4          4.0    Diana  60000.0  Senior Developer       60000       85000   
5          NaN      NaN      NaN          Director      100000      150000   

       _merge  
0        both  
1        both  
2        both  
3        both  
4        both  
5  right_only  


### Performance Optimization with Numba
For large datasets, you can significantly speed up computations by enabling Numba (if installed):

In [11]:
# Enable Numba optimization (requires numba package)
result_optimized = employees.conditional_join(
    jobs,
    ('salary', 'min_salary', '>='), 
    ('salary', 'max_salary', '<='),
    use_numba=True
)

###  Date Range Analysis
Here's another practical example: finding events that occurred during specific time periods.

In [12]:
# Sample  website visits and marketing campaigns
visits = pd.DataFrame({
    'user_id': [1, 2, 3, 4, 5],
    'visit_time': pd.to_datetime([
        '2023-01-15 10:00:00',
        '2023-01-20 14:30:00',
        '2023-02-05 09:15:00',
        '2023-02-10 16:20:00',
        '2023-02-25 11:45:00'
    ])
})

campaigns = pd.DataFrame({
    'campaign_name': ['Winter Sale', 'New Year Promo', 'February Frenzy'],
    'start_date': pd.to_datetime(['2023-01-10', '2023-01-15', '2023-02-01']),
    'end_date': pd.to_datetime(['2023-01-25', '2023-01-31', '2023-02-28'])
})

# Find which campaign each visit occurred during
visit_campaigns = visits.conditional_join(
    campaigns,
    ('visit_time', 'start_date', '>='),  # visit_time >= campaign_start
    ('visit_time', 'end_date', '<=')     # visit_time <= campaign_end
)

print("\nWebsite visits matched with campaigns:")
print(visit_campaigns)


Website visits matched with campaigns:
   user_id          visit_time    campaign_name start_date   end_date
0        1 2023-01-15 10:00:00      Winter Sale 2023-01-10 2023-01-25
1        1 2023-01-15 10:00:00   New Year Promo 2023-01-15 2023-01-31
2        2 2023-01-20 14:30:00      Winter Sale 2023-01-10 2023-01-25
3        2 2023-01-20 14:30:00   New Year Promo 2023-01-15 2023-01-31
4        3 2023-02-05 09:15:00  February Frenzy 2023-02-01 2023-02-28
5        4 2023-02-10 16:20:00  February Frenzy 2023-02-01 2023-02-28
6        5 2023-02-25 11:45:00  February Frenzy 2023-02-01 2023-02-28


## Summary and Conclusion

In this notebook, we explored data wrangling using the `pyjanitor` library in Python. We covered key functions such as `clean_names()`, `get_dupes()`, and `remove_empty()`, demonstrating how they can simplify common data cleaning tasks. By applying these functions to a real-world dataset, we illustrated a step-by-step approach to transforming messy data into a tidy format. The integration of `pyjanitor` with pandas allows for intuitive and efficient data manipulation, making it a valuable tool for data scientists and analysts. Overall, `pyjanitor` enhances the data cleaning process, enabling users to focus more on analysis and insights.

##  Resources

- [pyjanitor Documentation](https://pyjanitor.readthedocs.io/)
- [pandas CROSSTAB Docs](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html)
- [R janitor Vignette](https://cran.r-project.org/web/packages/janitor/vignettes/janitor.html)
- [Tidyverse Style Guide](https://style.tidyverse.org/) → Applies to Python too!